# MNIST with CNN

In [1]:
import tensorflow.compat.v1 as tf
tf.enable_eager_execution()
import tensorflow_datasets as tfds

In [2]:
# pre written class to replace outdated tensorflow code
class MyDS(object):
  class SubDS(object):
      import numpy as np
      def __init__(self, ds, *, one_hot):
          np = self.__class__.np
          self.ds = [e for e in ds.as_numpy_iterator()]
          self.sds = {(k + 's') : np.stack([
              (e[k] if len(e[k].shape) > 0 else e[k][None]).reshape(-1) for e in self.ds
          ], 0) for k in self.ds[0].keys()}
          self.one_hot = one_hot
          if one_hot is not None:
              self.max_one_hot = np.max(self.sds[one_hot + 's'])
      def _to_one_hot(self, a, maxv):
          np = self.__class__.np
          na = np.zeros((a.shape[0], maxv + 1), dtype = a.dtype)
          for i, e in enumerate(a[:, 0]):
              na[i, e] = True
          return na
      def _apply_one_hot(self, key, maxv):
          assert maxv >= self.max_one_hot, (maxv, self.max_one_hot)
          self.max_one_hot = maxv
          self.sds[key + 's'] = self._to_one_hot(self.sds[key + 's'], self.max_one_hot)
      def next_batch(self, num = 16):
          np = self.__class__.np
          idx = np.random.choice(len(self.ds), num)
          res = {k : np.stack([
              (self.ds[i][k] if len(self.ds[i][k].shape) > 0 else self.ds[i][k][None]).reshape(-1) for i in idx
          ], 0) for k in self.ds[0].keys()}
          if self.one_hot is not None:
              res[self.one_hot] = self._to_one_hot(res[self.one_hot], self.max_one_hot)
          for i, (k, v) in enumerate(list(res.items())):
              res[i] = v
          return res
      def __getattr__(self, name):
          if name not in self.__dict__['sds']:
              return self.__dict__[name]
          return self.__dict__['sds'][name]
  def __init__(self, name, *, one_hot = None):
      self.ds = tfds.load(name)
      self.sds = {}
      for k, v in self.ds.items():
          self.sds[k] = self.__class__.SubDS(self.ds[k], one_hot = one_hot)
      if one_hot is not None:
          maxh = max(e.max_one_hot for e in self.sds.values())
          for e in self.sds.values():
              e._apply_one_hot(one_hot, maxh)
  def __getattr__(self, name):
      if name not in self.__dict__['sds']:
          return self.__dict__[name]
      return self.__dict__['sds'][name]

In [3]:
mnist = MyDS('mnist', one_hot = 'label') # tensorflow_datasets.load('mnist') doesn't work anymore

tf.disable_eager_execution()
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def get_weights(shape):
    data = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(data)

In [5]:
def get_biases(shape):
    data = tf.constant(0.1, shape=shape)
    return tf.Variable(data)

In [6]:
def create_layer(shape):
    W = get_weights(shape)
    b = get_biases([shape[-1]])
    return W, b

In [7]:
def convolution_2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [8]:
def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
x = tf.placeholder(tf.float32, [None, 784])

In [10]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [11]:
W_conv1, b_conv1 = create_layer([5, 5, 1, 32])

In [12]:
h_conv1 = tf.nn.relu(convolution_2d(x_image, W_conv1) + b_conv1)

In [13]:
h_pool1 = max_pooling(h_conv1)

In [14]:
W_conv2, b_conv2 = create_layer([5, 5, 32, 64])

In [15]:
h_conv2 = tf.nn.relu(convolution_2d(h_pool1, W_conv2) + b_conv2)

In [16]:
h_pool2 = max_pooling(h_conv2)

In [17]:
W_fc1, b_fc1 = create_layer([7 * 7 * 64, 1024])

In [18]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

In [19]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [20]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
W_fc2, b_fc2 = create_layer([1024, 10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [22]:
y_loss = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_loss, logits=y_conv))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [23]:
predicted = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_loss, 1))
accuracy = tf.reduce_mean(tf.cast(predicted, tf.float32))

In [24]:
sess = tf.InteractiveSession()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [25]:
num_iterations = 200
batch_size = 75
print('\nTraining the model....')
for i in range(num_iterations):
    batch = mnist.train.next_batch(batch_size)

    if i % 50 == 0:
        cur_accuracy = accuracy.eval(feed_dict = {
                x: batch[0], y_loss: batch[1], keep_prob: 1.0})
        print('Iteration', i, ', Accuracy =', cur_accuracy)

    optimizer.run(feed_dict = {x: batch[0], y_loss: batch[1], keep_prob: 0.5})

print('Test accuracy =', accuracy.eval(feed_dict = {
        x: mnist.test.images, y_loss: mnist.test.labels,
        keep_prob: 1.0}))


Training the model....
Iteration 0 , Accuracy = 0.013333334
Iteration 50 , Accuracy = 0.85333335
Iteration 100 , Accuracy = 0.8
Iteration 150 , Accuracy = 0.93333334
Test accuracy = 0.9162


## Great Job!